## Using SOFIE with RDataFrame in Python

Notebook derived from the tutorial [TMVA_SOFIE_RDataFrame.py](https://root.cern.ch/doc/master/TMVA__SOFIE__RDataFrame_8py.html)

In this example we are going to start from the Keras input model file and use SOFIE and then the ROOT JIT to compile the model.

In [ ]:
import ROOT

In [ ]:
modelFile = "Higgs_trained_model.h5"
if (ROOT.gSystem.AccessPathName(modelFile)) :
    ROOT.Info("TMVA_SOFIE_RDataFrame","You need to run TMVA_Higgs_Classification to generate the Keras trained model")
    exit()


### Parse input file (directly from Keras)

Parse the input file using the direct Keras to RModel converter

In [ ]:
# parse the input Keras model into RModel object
model = ROOT.TMVA.Experimental.SOFIE.PyKeras.Parse(modelFile)

In [ ]:
#Generating inference code
model.Generate()
model.OutputGenerated()
model.PrintGenerated()

### Compile generated model using ROOT JIT

In [ ]:
# now compile using ROOT JIT trained model
print("compiling SOFIE model and functor....")
generatedFile = "Higgs_trained_model.hxx"
modelName = "Higgs_trained_model"
ROOT.gInterpreter.Declare('#include "' + generatedFile + '"')
ROOT.gInterpreter.Declare('auto sofie_functor = TMVA::Experimental::SofieFunctor<7,TMVA_SOFIE_'+modelName+'::Session>(0);')


### Use model with RDataFrame

In [ ]:
inputFile = "http://root.cern.ch/files/Higgs_data.root"

In [ ]:
df1 = ROOT.RDataFrame("sig_tree", inputFile)
h1 = df1.Define("DNN_Value", "sofie_functor(rdfslot_,m_jj, m_jjj, m_lv, m_jlv, m_bb, m_wbb, m_wwbb)").Histo1D(("h_sig", "", 100, 0, 1),"DNN_Value")

df2 = ROOT.RDataFrame("bkg_tree", inputFile)
h2 = df2.Define("DNN_Value", "sofie_functor(rdfslot_,m_jj, m_jjj, m_lv, m_jlv, m_bb, m_wbb, m_wwbb)").Histo1D(("h_bkg", "", 100, 0, 1),"DNN_Value")

In [ ]:
h1.SetLineColor(ROOT.kRed)
h2.SetLineColor(ROOT.kBlue)

c1 = ROOT.TCanvas()
ROOT.gStyle.SetOptStat(0)

h2.Draw()
h1.Draw("SAME")
c1.Draw()